In [1]:
import ee 
import geemap
import eemont
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
ee.Authenticate()
ee.Initialize() 


Successfully saved authorization token.


In [3]:
# site coordinates
xy = [134.6, 67.66]
poi = ee.Geometry.Point(xy)
# create map
Map = geemap.Map()
Map.addLayer(poi)
Map.centerObject(poi, zoom=3)
Map

Map(center=[67.66, 134.6], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

In [19]:
point = ee.Geometry.PointFromQuery(
    'Cali, Colombia',
    user_agent = 'eemont-example'
) # Extended constructor
S2 = (ee.ImageCollection("COPERNICUS/S2_SR")
        .filterBounds(point)
        .filterDate("2020-01-01","2021-01-01")
        .preprocess()
        .maskClouds()
        .scaleAndOffset()
        )
S2online = S2.spectralIndices(["NDVI","kNDVI","NDWI","SeLI"],online = True).median()

Exception: [p] and [c] must be positive! Values passed: p = False, c = 1.0

In [16]:
point = ee.Geometry.PointFromQuery(
    'Cali, Colombia',
    user_agent = 'eemont-example'
) # Extended constructor
S2 = (ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(point)
    .closest('2020-10-15') # Extended (pre-processing)
    .maskClouds(prob = 70) # Extended (pre-processing)
    .scaleAndOffset() # Extended (pre-processing)
    .spectralIndices(['NDVI','NDWI','BAIS2'])) # Extended (processing)

Exception: [p] and [c] must be positive! Values passed: p = False, c = 1.0

In [ ]:
S2 = ee.ImageCollection('COPERNICUS/S2_SR').scaleAndOffset()
S2withIndices = S2.spectralIndices()

In [31]:
# load image
S2_Image = (ee.Image('COPERNICUS/S2_SR/20210805T030551_20210805T030545_T53WMR')
            .scaleAndOffset()
            .spectralIndices(['NDVI', 'NBR']) # Calculate multispectral indices
           )


# S2_Image = ((ee.ImageCollection('COPERNICUS/S2_SR').first())
#             .scaleAndOffset()
#             .spectralIndices(['NDVI', 'NBR']) # Calculate multispectral indices
#            )

# Show
Map.addLayer(S2_Image, dict(min=0.0, max=0.15, gamma=1.5, bands=['B2', 'B3', 'B2']), 'RGB')
Map.addLayer(S2_Image, dict(min=0.0, max=0.3, gamma=1.5, bands=['B8', 'B4', 'B3']), 'Color-Infrared', False)

RdYlGn = ['#d7191c','#fdae61','#ffffbf','#a6d96a','#1a9641']
Map.addLayer(S2_Image, dict(min=0.0, max=0.8, bands=['NDVI'], palette=RdYlGn), 'NDVI')
Map.addLayer(S2_Image, dict(min=0.0, max=0.8, bands=['NBR'], palette=RdYlGn), 'NBR')

Map.centerObject(S2_Image, zoom=8)

Map

Exception: [p] and [c] must be positive! Values passed: p = False, c = 1.0

# Code that works not in the book

In [21]:
import ee 
import geemap
import eemont
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [27]:
bbox = ee.Geometry.BBox(134.5, 67.7, 134.8, 67.597)
Map = geemap.Map()
Map.addLayer(bbox)
Map.centerObject(bbox, zoom=10)
Map

Map(center=[67.64848549607278, 134.65000000000018], controls=(WidgetControl(options=['position', 'transparent_…

In [28]:
# Image Collection
ds = 'COPERNICUS/S2_SR'

S2 = (ee.ImageCollection(ds)
      ### Filters ###
      .filterBounds(poi) # only images, which intersect our point/region
      .filter(ee.Filter.calendarRange(6,9,'month')) # only June through September, Summer only
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) # only images with less that 10 % clouds
      #.filterDate('2022-07-01','2022-12-31')
      ### PreProcessing ###
      .scaleAndOffset()
      .spectralIndices(['NDVI', 'NBR'])
      .maskClouds()
      .sort('CLOUDY_PIXEL_PERCENTAGE')
     )#.first()
# calculate indices (e.g. NDVI, NBR)
print(f'Number of images {S2.size().getInfo()}') # Print number of images

Exception: [p] and [c] must be positive! Values passed: p = False, c = 1.0

In [ ]:
#Map = geemap.Map()
Map.addLayer(S2, dict(min=0.0, max=0.15, gamma=1.5, bands=['B2', 'B3', 'B2']), 'RGB_Collection')
Map.addLayer(S2, dict(min=0.0, max=0.3, gamma=1.5, bands=['B8', 'B4', 'B3']), 'Color-Infrared_Collection')
RdYlGn = ['#d7191c','#fdae61','#ffffbf','#a6d96a','#1a9641']

# Show NDVI
#Map.addLayer(S2_Image, dict(min=0.0, max=0.8, bands=['NDVI'], palette=RdYlGn), 'NDVI')
# Show NDVI 
#Map.addLayer(S2_Image, dict(min=0.0, max=0.8, bands=['NBR'], palette=RdYlGn), 'NBR')
Map.centerObject(S2_Image, zoom=8)
Map

## Timeseries

In [29]:
def load_and_prepare_Landsat(collection_name):
    collection = (ee.ImageCollection(collection_name)
      ### Filters ###
      .filterBounds(poi) # only images, which intersect our point/region
      .filter(ee.Filter.calendarRange(7,8,'month')) # only June through September, Summer only
      .filter(ee.Filter.lt('CLOUD_COVER_LAND', 10)) # only images with less that 10 % clouds
      #.filterDate('2022-07-01','2022-12-31')
      ### PreProcessing ###
      .scaleAndOffset()
      .spectralIndices(['NDVI', 'NBR'])
      .maskClouds()
     )
    return collection

In [30]:
L9 = load_and_prepare_Landsat("LANDSAT/LC09/C02/T1_L2")
L8 = load_and_prepare_Landsat("LANDSAT/LC08/C02/T1_L2")
L7 = load_and_prepare_Landsat("LANDSAT/LE07/C02/T1_L2")
L5 = load_and_prepare_Landsat("LANDSAT/LT05/C02/T1_L2")
LS = L9.merge(L8).merge(L7).merge(L5)

Exception: [p] and [c] must be positive! Values passed: p = False, c = 1.0